# Predicting Traffic Volume with PyTorch

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

In [2]:
# Read in the data
train_data = pd.read_csv('data/train_scaled.csv')
test_data = pd.read_csv('data/test_scaled.csv')

# View the results
train_data.head()
test_data.head()

,temp,rain_1h,snow_1h,clouds_all,holiday_Christmas Day,holiday_Columbus Day,holiday_Independence Day,holiday_Labor Day,holiday_Martin Luther King Jr Day,holiday_Memorial Day,...,weather_description_thunderstorm with heavy rain,weather_description_thunderstorm with light drizzle,weather_description_thunderstorm with light rain,weather_description_thunderstorm with rain,weather_description_very heavy rain,hour_of_day,day_of_week,day_of_month,month,traffic_volume
0,0.808980,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.203022
1,0.808072,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.193407
2,0.807358,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086957,0.0,0.0,0.0,0.127885
3,0.806904,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.130435,0.0,0.0,0.0,0.080357
4,0.805963,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.173913,0.0,0.0,0.0,0.052335


In [3]:
# Convert the data from DataFrames to NumPy arrays
train_array = train_data.to_numpy()
test_array = test_data.to_numpy()

In [4]:
# Function to create sequences from the data
# This is for preparing the data to predict future values using past data (time-series)
def create_sequences(data, sequence_length, target_column):
    """
    This function creates input sequences and the corresponding target values
    based on the given sequence length and target column.
    
    Arguments:
    data -- The data in NumPy array format.
    sequence_length -- How many past time steps we use to predict the next one.
    target_column -- The column that holds the target value we want to predict.
    
    Returns:
    sequences -- The input sequences of past data.
    targets -- The corresponding target values to predict.
    """
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]  # Get the sequence of past data
        target = data[i + sequence_length, target_column]  # Get the target value
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

In [5]:
# Create sequences for both training and testing data
sequence_length = 12  # We'll use 12 past data points to predict the next one
target_column = -1  # Assuming the target is the last column (change this if needed)

X_train, y_train = create_sequences(train_array, sequence_length, target_column)
X_test, y_test = create_sequences(test_array, sequence_length, target_column)

In [6]:
# Convert the data into PyTorch tensors and wrap it in a dataset
train_dataset = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.float32)
)
test_dataset = TensorDataset(
    torch.tensor(X_test, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.float32)
)

In [7]:
# Create DataLoader to load the data in batches (shuffling for training, no shuffle for testing)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [8]:
# Define the LSTM model class
class TrafficVolumeModel(nn.Module):
    def __init__(self):
        super(TrafficVolumeModel, self).__init__()
        # LSTM layer: Takes in data, processes it, and produces hidden states
        self.lstm = nn.LSTM(input_size=66, hidden_size=64, num_layers=2, batch_first=True)
        # Fully connected layer to output a single prediction
        self.fc = nn.Linear(64, 1)
        # ReLU activation to add non-linearity
        self.relu = nn.LeakyReLU()

    def forward(self, x):
        # Pass the input data through the LSTM layer
        _, (h_n, _) = self.lstm(x)
        # Use the last hidden state to make a prediction
        out = self.fc(h_n[-1])
        return self.relu(out)


In [9]:
# Initialize the model
model = TrafficVolumeModel()

# Define the loss function (Mean Squared Error) and the optimizer (Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model for a few epochs
epochs = 2
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()  # Reset the gradients before the new forward pass
        predictions = model(batch_x)  # Make predictions
        loss = criterion(predictions, batch_y)  # Calculate loss
        loss.backward()  # Calculate gradients
        optimizer.step()  # Update the model's parameters

    print(f"Epoch: {epoch+1}, Training Loss: {loss.item():.5f}")


/home/workbench/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/workbench/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([46])) that is different to the input size (torch.Size([46, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1, Training Loss: 0.06049
Epoch: 2, Training Loss: 0.10027


In [10]:
# Set the model to evaluation mode to test it
model.eval()

# Gather predictions and true values from the test set
all_predictions = []
all_labels = []

In [11]:
# Turn off gradients during testing to speed things up
with torch.no_grad():
    for seqs, labels in test_loader:
        preds = model(seqs).squeeze()  # Make predictions and remove extra dimensions
        all_predictions.append(preds)
        all_labels.append(labels)

In [12]:
# Combine all predictions and labels into one big tensor
all_predictions = torch.cat(all_predictions)
all_labels = torch.cat(all_labels)

# Calculate the Mean Squared Error (MSE) for the test data
test_mse = F.mse_loss(all_predictions, all_labels)
print(f'Test MSE: {test_mse.item():.5f}')

Test MSE: 0.07372
